In [2]:
import mysql.connector
 
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Yankejing020711"   #REPLACE THIS WITH THE PASSWORD YOU SET
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")

CONNECTION SUCCESSFUL


In [20]:
#create a database
mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE project1_test")

DatabaseError: 1007 (HY000): Can't create database 'project1_test'; database exists

In [3]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Yankejing020711", #REPLACE THIS WITH YOUR PASSWORD 
  database = "project1_test" #connecting to testDatabase
)

#redo table
mycursor = mydb.cursor()

In [149]:
#redo table
mycursor = mydb.cursor()

mycursor.execute("DROP TABLE IF EXISTS patients, physicians, pharmacies, drugs, prescriptions, adverse_interactions, alerts, pharmacy_fills, companies, contracts;")

In [150]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1_test.physicians(
        ssn  varchar(11),
        name varchar(128),
        primary_specialty  varchar(256),
        experience_years  decimal(5,2),
        PRIMARY KEY (ssn)
    )
    '''
)

In [151]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1_test.patients(
        ssn  varchar(11),
        name  varchar(128),
        address  varchar(256),
        birthdate  date,
        physician_id  varchar(11),
        PRIMARY KEY (ssn),
        FOREIGN KEY (physician_id) REFERENCES physicians (ssn)
    )
    '''
)

In [152]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1_test.pharmacies(
        id  varchar(128),
        name  varchar(256),
        address varchar(256),
        phone  varchar(20),
        PRIMARY KEY (id)
    )
    '''
)

In [153]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1_test.drugs(
        id  varchar(20),
        name  varchar(256),
        PRIMARY KEY (name)
    )
    '''
)

In [154]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1_test.prescriptions(
        id  varchar(20),
        patient_id  varchar(11),
        physician_id  varchar(11),
        drug_name  varchar(256),
        date  date,
        quantity  int,
        PRIMARY KEY (id),
        FOREIGN KEY (patient_id) REFERENCES patients (ssn),
        FOREIGN KEY (physician_id) REFERENCES physicians (ssn),
        FOREIGN KEY (drug_name) REFERENCES drugs (name)
    )
    '''
)

In [155]:
mycursor.execute("CREATE INDEX idx_patient_drug ON project1_test.prescriptions(patient_id, drug_name)")

In [156]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1_test.adverse_interactions(
        drug_name  varchar(256),
        drug_name_2  varchar(256),
        PRIMARY KEY (drug_name, drug_name_2),
        FOREIGN KEY (drug_name) REFERENCES drugs (name),
        FOREIGN KEY (drug_name_2) REFERENCES drugs (name)
    )
    '''
)

In [157]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TABLE project1_test.alerts(
        patient_id  varchar(11),
        physician_id  varchar(11),
        alert_date  date,
        drug1  varchar(256),
        drug2  varchar(256),
        PRIMARY KEY (patient_id, physician_id, alert_date, drug1, drug2),
        FOREIGN KEY (patient_id) REFERENCES patients (ssn),
        FOREIGN KEY (physician_id) REFERENCES physicians (ssn),
        FOREIGN KEY (patient_id, drug1) REFERENCES prescriptions (patient_id, drug_name),
        FOREIGN KEY (patient_id, drug2) REFERENCES prescriptions (patient_id, drug_name)
    )
    '''
)

In [171]:
#redo table
mycursor = mydb.cursor()

mycursor.execute("DROP TRIGGER IF EXISTS alerts;")

In [172]:
mycursor = mydb.cursor()

mycursor.execute(
    '''
    CREATE TRIGGER alerts
    AFTER INSERT ON project1_test.prescriptions
    FOR EACH ROW
    BEGIN
        INSERT INTO project1_test.alerts (patient_id, physician_id, alert_date, drug1, drug2)
        SELECT DISTINCT p2.patient_id, NEW.physician_id, NEW.date, p2.drug_name, NEW.drug_name
        FROM project1_test.prescriptions p2
        INNER JOIN project1_test.adverse_interactions ai ON
            (ai.drug_name = p2.drug_name AND ai.drug_name_2 = NEW.drug_name)
            OR (ai.drug_name = NEW.drug_name AND ai.drug_name_2 = p2.drug_name)
        WHERE p2.patient_id = NEW.patient_id;
    END;
    '''
)

In [142]:
import pandas as pd

In [165]:
data = pd.read_csv("physicians.csv")
for i,row in data.iterrows():
            sql = "INSERT INTO project1_test.physicians VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [166]:
data = pd.read_csv("patients.csv")
data['birthdate'] = pd.to_datetime(data['birthdate']).dt.date
for i,row in data.iterrows():
            sql = "INSERT INTO project1_test.patients VALUES (%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [167]:
data = pd.read_csv("pharmacies.csv")
for i,row in data.iterrows():
            sql = "INSERT INTO pharmacies VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [168]:
data = pd.read_csv("drugs.csv")

for i,row in data.iterrows():
            sql = "INSERT INTO project1_test.drugs VALUES (%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

query = "SELECT * from prescriptions"
mycursor.execute(query)

for x in mycursor:
    print(x)

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [169]:
data = pd.read_csv("adverse_reactions.csv")

for i,row in data.iterrows():
            sql = "INSERT INTO project1_test.adverse_interactions VALUES (%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

query = "SELECT * from adverse_interactions"
mycursor.execute(query)

for x in mycursor:
    print(x)

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
('Primalovir', 'Abnazole Toleluble')
('Cleotrana', 'Avafoxin')
('Quixiposide', 'Avafoxin')
('Quixiposide', 'Dantopex Quixilinum')
('Olanzanafine', 'Glucozepam Amcipentin')
('Avafoxin', 'Kanulin')
('Cleotrana', 'Kanulin')
('Primalovir', 'Olanzanafine')
('Cleotrana', 'Quixiposide')


In [173]:
data = pd.read_csv("prescriptions.csv")
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

for i,row in data.iterrows():
            sql = "INSERT INTO prescriptions VALUES (%s,%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

query = "SELECT * from prescriptions"
mycursor.execute(query)

for x in mycursor:
    print(x)

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
('1', '478-34-0781', '614-57-6885', 'Avafoxin', datetime.date(2023, 3, 11), 90)
('10', '501-47-2038', '156-28-1945', 'Cleotrana', datetime.date(2023, 7, 18), 20)
('11', '777-39-3296', '718-27-0905', 'Dantopex Quixilinum', datetime.date(2023, 8, 2), 1)
('12', '501-47-2038', '156-28-1945', 'Cleotrana', datetime.date(2023, 8, 21), 10)
('13', '478-34-0781', '614-57-6885', 'Avafoxin', datetime.date(2023, 9, 17), 30)
('14', '478-34-0781', '614-57-6885', 'Quixiposide', datetime.date(2023, 9, 17), 2)
('15', '501-47-2038', '156-28-1945', 'Avafoxin', datetime.date(2023, 9, 22), 15)
('16', '501-47-2038', '156-28-1945', 'Kanulin', datetime.date(2023, 9, 22), 8)
('2', '758-08-7274', '118-66-5958', 'Cleotrana', datetime.date(2023, 3, 12), 10)
('3',